In [101]:
import time
import sys
import struct
import io

In [102]:
# Tag Types
tyEmpty8 = struct.unpack(">i", bytes.fromhex("FFFF0008"))[0]
tyBool8 = struct.unpack(">i", bytes.fromhex("00000008"))[0]
tyInt8 = struct.unpack(">i", bytes.fromhex("10000008"))[0]
tyBitSet64 = struct.unpack(">i", bytes.fromhex("11000008"))[0]
tyColor8 = struct.unpack(">i", bytes.fromhex("12000008"))[0]
tyFloat8 = struct.unpack(">i", bytes.fromhex("20000008"))[0]
tyColor8 = struct.unpack(">i", bytes.fromhex("21000008"))[0]
tyFloat8Array = struct.unpack(">i", bytes.fromhex("2001FFFF"))[0]
tyAnsiString = struct.unpack(">i", bytes.fromhex("4001FFFF"))[0]
tyWideString = struct.unpack(">i", bytes.fromhex("4002FFFF"))[0]
tyBinaryBlob = struct.unpack(">i", bytes.fromhex("FFFFFFFF"))[0]
tyTDateTime = struct.unpack(">i", bytes.fromhex("21000008"))[0]

In [103]:
print(tyEmpty8,tyBool8,tyInt8,tyBitSet64,tyFloat8,tyColor8,tyFloat8Array, tyAnsiString, tyWideString, tyBinaryBlob)

-65528 8 268435464 285212680 536870920 553648136 537001983 1073872895 1073938431 -1


In [104]:
# Record types
rtPicoHarpT3 = struct.unpack(">i", bytes.fromhex("00010303"))[0]
rtPicoHarpT2 = struct.unpack(">i", bytes.fromhex("00010203"))[0]
rtHydraHarpT3 = struct.unpack(">i", bytes.fromhex("00010304"))[0]
rtHydraHarpT2 = struct.unpack(">i", bytes.fromhex("00010204"))[0]
rtHydraHarp2T3 = struct.unpack(">i", bytes.fromhex("01010304"))[0]
rtHydraHarp2T2 = struct.unpack(">i", bytes.fromhex("01010204"))[0]
rtTimeHarp260NT3 = struct.unpack(">i", bytes.fromhex("00010305"))[0]
rtTimeHarp260NT2 = struct.unpack(">i", bytes.fromhex("00010205"))[0]
rtTimeHarp260PT3 = struct.unpack(">i", bytes.fromhex("00010306"))[0]
rtTimeHarp260PT2 = struct.unpack(">i", bytes.fromhex("00010206"))[0]
rtMultiHarpT3 = struct.unpack(">i", bytes.fromhex("00010307"))[0]
rtMultiHarpT2 = struct.unpack(">i", bytes.fromhex("00010207"))[0]

In [105]:
# global variables
global inputfile
global outputfile
global recNum
global oflcorrection
global truensync
global dlen
global isT2
global globRes
global numRecords

In [106]:
fname="/Users/jjgomezcadenas/Projects/LaserLab/test/test.ptu"

In [107]:
global inputfile
global outputfile
global recNum
global oflcorrection
global truensync
global dlen
global isT2
global globRes
global numRecords

In [109]:

inputfile = open(fname, "rb")
# The following is needed for support of wide strings
outputfile = io.open("/Users/jjgomezcadenas/Projects/LaserLab/test/out.txt", "w+")

magic = inputfile.read(8).decode("utf-8").strip("\0")
if magic != "PQTTTR":
    print("ERROR: Magic invalid, this is not a PTU file.")
    inputfile.close()
    outputfile.close()
    exit(0)

version = inputfile.read(8).decode("utf-8").strip("\0")
outputfile.write("Tag version: %s\n" % version)

# Write the header data to outputfile and also save it in memory.
# There's no do ... while in Python, so an if statement inside the while loop
# breaks out of it
tagDataList = []  # Contains tuples of (tagName, tagValue)
# Write the header data to outputfile and also save it in memory.
# There's no do ... while in Python, so an if statement inside the while loop
# breaks out of it


i = 0
while True:
#while i < 20:
    i+=1
    tagIdent = inputfile.read(32).decode("utf-8").strip("\0")
    tagIdx = struct.unpack("<i", inputfile.read(4))[0]
    tagTyp = struct.unpack("<i", inputfile.read(4))[0]
    if tagIdx > -1:
        evalName = tagIdent + "(" + str(tagIdx) + ")"
    else:
        evalName = tagIdent
    outputfile.write("\n%-40s" % evalName)
    
    #print("Reading cycle = ", i)
    #print("tagType =", tagTyp)
    
    if tagTyp == tyEmpty8:
        tagInt = inputfile.read(8)
        outputfile.write("<empty Tag>")
        tagDataList.append((evalName, "<empty Tag>"))
        
        #print("case tyEmpty8: tagInt =", tagInt)
        
    elif tagTyp == tyBool8:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        if tagInt == 0:
            outputfile.write("False")
            tagDataList.append((evalName, "False"))
        else:
            outputfile.write("True")
            tagDataList.append((evalName, "True"))
            
        #print("case tyBool8: tagInt =", tagInt)
        
    elif tagTyp == tyInt8:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        outputfile.write("%d" % tagInt)
        tagDataList.append((evalName, tagInt))
        
        #print("case tyInt8: tagInt =", tagInt)
        
    elif tagTyp == tyBitSet64:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        outputfile.write("{0:#0{1}x}".format(tagInt, 18))
        tagDataList.append((evalName, tagInt))
        
        #print("case tyBitSet64: tagInt =", tagInt)
        
    elif tagTyp == tyColor8:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        outputfile.write("{0:#0{1}x}".format(tagInt, 18))
        tagDataList.append((evalName, tagInt))
        
        #print("case tyColor8: tagInt =", tagInt)
        
    elif tagTyp == tyFloat8:
        tagFloat = struct.unpack("<d", inputfile.read(8))[0]
        outputfile.write("%-3E" % tagFloat)
        tagDataList.append((evalName, tagFloat))
        
        #print("case tyFloat8: tagFloat =", tagFloat)
        
    elif tagTyp == tyFloat8Array:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        outputfile.write("<Float array with %d entries>" % tagInt / 8)
        tagDataList.append((evalName, tagInt))
        
        #print("case tyFloat8Array: tagInt =", tagInt)
        
    elif tagTyp == tyTDateTime:
        tagFloat = struct.unpack("<d", inputfile.read(8))[0]
        tagTime = int((tagFloat - 25569) * 86400)
        tagTime = time.gmtime(tagTime)
        #print("tagTime =", tagTime)
        outputfile.write(time.strftime("%a %b %d %H:%M:%S %Y", tagTime))
        tagDataList.append((evalName, tagTime))
        
        #print("case tyTDateTime: tagTime =", tagTime)
        
    elif tagTyp == tyAnsiString:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        tagString = inputfile.read(tagInt).decode("utf-8").strip("\0")
        outputfile.write("%s" % tagString)
        tagDataList.append((evalName, tagString))
        
        #print("case tyAnsiString: tagString =", tagString)
        
    elif tagTyp == tyWideString:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        tagString = (
            inputfile.read(tagInt).decode("utf-16le", errors="ignore").strip("\0")
        )
        outputfile.write(tagString)
        tagDataList.append((evalName, tagString))
        
        #print("case tyWideString: tagString =", tagString)
        
    elif tagTyp == tyBinaryBlob:
        tagInt = struct.unpack("<q", inputfile.read(8))[0]
        outputfile.write("<Binary blob with %d bytes>" % tagInt)
        tagDataList.append((evalName, tagInt))
        
        #print("case tyBinaryBlob: tagInt =", tagInt)
    else:
        print("ERROR: Unknown tag type")
        #exit(0)
    
    
    if tagIdent == "Header_End":
        break
        
print("tagDataList =", tagDataList)

tagDataList = [('File_GUID', '{C2A0FD07-1212-47FE-9799-6DEADA712E69}'), ('File_AssuredContent', 'HydraHarp: HWSETG SWSETG'), ('CreatorSW_ContentVersion', '3.0'), ('CreatorSW_Name', 'HydraHarp AcqUI'), ('CreatorSW_Version', '3.0.0.4'), ('File_CreatingTime', 4676432107605872482), ('File_Comment', 'T3 Mode'), ('Measurement_Mode', 3), ('Measurement_SubMode', 0), ('TTResult_StopReason', 1), ('Fast_Load_End', '<empty Tag>'), ('TTResultFormat_TTTRRecType', 16843524), ('TTResultFormat_BitsPerRecord', 32), ('MeasDesc_BinningFactor', 128), ('MeasDesc_Offset', 0), ('MeasDesc_AcquisitionTime', 60000), ('MeasDesc_StopAt', 4294967295), ('MeasDesc_StopOnOvfl', 'True'), ('MeasDesc_Restart', 'True'), ('CurSWSetting_DispLog', 'True'), ('CurSWSetting_DispAxisTimeFrom', 0), ('CurSWSetting_DispAxisTimeTo', 250), ('CurSWSetting_DispAxisCountFrom', 10), ('CurSWSetting_DispAxisCountTo', 429295), ('CurSWSetting_DispCurves', 8), ('CurSWSetting_DispCurve_MapTo(0)', 0), ('CurSWSetting_DispCurve_Show(0)', 'True'),

In [110]:
def readHT3(version):
    global inputfile, outputfile, recNum, oflcorrection, numRecords
    T3WRAPAROUND = 1024
    #for recNum in range(0, numRecords):
    for recNum in range(0, 10):
        try:
            recordData = "{0:0{1}b}".format(
                struct.unpack("<I", inputfile.read(4))[0], 32
            )
        except:
            print(
                "The file ended earlier than expected, at record %d/%d."
                % (recNum, numRecords)
            )
            exit(0)

        special = int(recordData[0:1], base=2)
        channel = int(recordData[1:7], base=2)
        dtime = int(recordData[7:22], base=2)
        nsync = int(recordData[22:32], base=2)
        
        print("special = ", special)
        print("channel = ", channel)
        print("dtime = ", dtime)
        print("nsync = ", nsync)
        
        if special == 1:
            if channel == 0x3F:  # Overflow
                # Number of overflows in nsync. If 0 or old version, it's an
                # old style single overflow
                if nsync == 0 or version == 1:
                    oflcorrection += T3WRAPAROUND
                    gotOverflow(1)
                    #print("Will call gotOverflow(1)") 
                    #print("oflcorrection =", oflcorrection) 
                else:
                    oflcorrection += T3WRAPAROUND * nsync
                    gotOverflow(nsync)
                    #print("Will call gotOverflow(nsync)") 
                    #print("oflcorrection =", oflcorrection) 
                    
            if channel >= 1 and channel <= 15:  # markers
                truensync = oflcorrection + nsync
                gotMarker(truensync, channel)
                #print("Will call gotMarker(truensync, channel)") 
                #print("truensync =", truensync) 
                
        else:  # regular input channel
            truensync = oflcorrection + nsync
            gotPhoton(truensync, channel, dtime)
            #print("Will call gotPhoton(truensync, channel, dtime)") 
            #print("truensync =", truensync) 
        if recNum % 100000 == 0:
            sys.stdout.write(
                "\rProgress: %.1f%%" % (float(recNum) * 100 / float(numRecords))
            )
            sys.stdout.flush()

In [111]:
def gotOverflow(count):
    global outputfile, recNum
    outputfile.write("%u OFL * %2x\n" % (recNum, count))


def gotMarker(timeTag, markers):
    global outputfile, recNum
    outputfile.write("%u MAR %2x %u\n" % (recNum, markers, timeTag))


def gotPhoton(timeTag, channel, dtime):
    global outputfile, isT2, recNum
    print("gotPhoton:")
    if isT2:
        outputfile.write(
            "%u CHN %1x %u %8.0lf\n"
            % (recNum, channel, timeTag, (timeTag * globRes * 1e12))
        )
        print("recNum, = ", recNum, " channel = ", channel, " timeTag = ", timeTag)
        print("resol, = ", (timeTag * globRes * 1e12))
    else:
        outputfile.write(
            "%u CHN %1x %u %8.0lf %10u\n"
            % (recNum, channel, timeTag, (timeTag * globRes * 1e9), dtime)
        )
        print("recNum, = ", recNum, " channel = ", channel, " timeTag = ", timeTag)
        print("resol, = ", (timeTag * globRes * 1e9), " dtime =", dtime)


In [112]:
tagNames = [tagDataList[i][0] for i in range(0, len(tagDataList))]
tagValues = [tagDataList[i][1] for i in range(0, len(tagDataList))]

In [113]:
numRecords = tagValues[tagNames.index("TTResult_NumberOfRecords")]
globRes = tagValues[tagNames.index("MeasDesc_GlobalResolution")]
TTTRTagRes = tagValues[tagNames.index("MeasDesc_Resolution")]  # dtime resolution

print("dtime resolution %.2E s" % TTTRTagRes)

print("Writing %d records, this may take a while..." % numRecords)

dtime resolution 1.28E-10 s
Writing 246151 records, this may take a while...


In [114]:
oflcorrection = 0
dlen = 0
outputfile.write("\n-----------------------\n")
recordType = tagValues[tagNames.index("TTResultFormat_TTTRRecType")]

In [115]:
if recordType == rtHydraHarp2T3:
    isT2 = False 
    print("HydraHarp V2 T3 data")
    outputfile.write("HydraHarp V2 T3 data\n")
    outputfile.write("\nrecord# chan   nsync truetime/ns dtime\n")
    readHT3(2)
   

HydraHarp V2 T3 data
special =  0
channel =  0
dtime =  1512
nsync =  55
gotPhoton:
recNum, =  0  channel =  0  timeTag =  55
resol, =  10999.824002815954  dtime = 1512
Progress: 0.0%special =  0
channel =  0
dtime =  1270
nsync =  59
gotPhoton:
recNum, =  1  channel =  0  timeTag =  59
resol, =  11799.811203020752  dtime = 1270
special =  0
channel =  0
dtime =  1153
nsync =  85
gotPhoton:
recNum, =  2  channel =  0  timeTag =  85
resol, =  16999.728004351928  dtime = 1153
special =  0
channel =  0
dtime =  1270
nsync =  113
gotPhoton:
recNum, =  3  channel =  0  timeTag =  113
resol, =  22599.638405785507  dtime = 1270
special =  0
channel =  0
dtime =  1173
nsync =  143
gotPhoton:
recNum, =  4  channel =  0  timeTag =  143
resol, =  28599.54240732148  dtime = 1173
special =  0
channel =  0
dtime =  544
nsync =  229
gotPhoton:
recNum, =  5  channel =  0  timeTag =  229
resol, =  45799.26721172461  dtime = 544
special =  0
channel =  0
dtime =  300
nsync =  243
gotPhoton:
recNum, =  6

In [116]:
inputfile.close()
outputfile.close()